## Installs

In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install pytesseract

## Imports

In [ ]:
from PIL import Image as im 
import pytesseract 
import numpy as np
from numpy import asarray
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import seaborn as sns
import pandas as pd
import sys
from datetime import date

## Upload model to notebook

In [ ]:
model = tf.keras.models.load_model("/home/linuxu/Desktop/folder/modelLoad")

In [ ]:
model = tf.keras.Sequential([
  layers.Flatten(input_shape=(180, 180,3)),
  layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
  layers.Dropout(0.2),
  layers.Dense(10, activation=tf.nn.softmax)
])

## Detect frame as yellow or red card by model prediction

In [ ]:
def detectCard(frame):
           crop = frame[300:480,200:380] # crop relevant part of frame   
           img_array = tf.keras.utils.img_to_array(crop)
           img_array = tf.expand_dims(img_array, 0)

           predictions = model.predict(img_array) # model predicts if the current frame is a card event or not
           score = tf.nn.softmax(predictions[0])
  
           if np.argmax(score) <=  5:
                pred = "card"
           else:
                pred = "other"
           if (100 * np.max(score)) > 90 and pred == "card":
               print("This image most likely belongs to cards with a {:.2f} percent confidence.".format(100 * np.max(score)))
               return True
           return False

## Detect game time by frame - using ocr maching learning algorithm

In [ ]:
def gameTimeSlicingOutput(output): # cleaning game clock output as numerical format (for example : MM:SS)
    clean_string = "" 
    for c in output: 
        if '0' <= c <= '9' or c == ':': 
            clean_string = clean_string + c 
    return clean_string 

In [ ]:
def gameTimeRecognize(current_frame): 
    crop_frame = frame[30:70, 100:200] # crop from current frame the game clock relevant part
    crop_arr = np.reshape(crop_frame, (40, 100, 3)) 
    image_from_arr = im.fromarray(crop_arr) 
    image_from_arr.save("/home/linuxu/Desktop/folder/Game_time.jpg") 
    game_time_image = im.open("/home/linuxu/Desktop/folder/Game_time.jpg") 
    game_minute = (pytesseract.image_to_string(game_time_image, config='--psm 6')) # here we get the game time output in string
    clean_minute = gameTimeSlicingOutput(game_minute) # call the cleaning function
    return clean_minute 

In [ ]:
def halfTimeClassification(frame,frameID):
    game_time_string = gameTimeRecognize(frame) # get cleaning game time in string
    if game_time_string is None or len(game_time_string) != 5: #check if game time in right format (to avoid mistakes of the tesseract algorithm)
     # in case the tesseract algorithm output is incorrent - we decided to classify the half time according the frame number   
        if frameID > 21500: 
            return 2
        else: return 1
        
    else: 
        current_minute = game_time_string[0]+ game_time_string[1]
        minute_int = int(current_minute)
        if minute_int <= 45 and frameID <= 21500:
            return 1
        else: return 2

## Create new class - Team

In [ ]:
class Team: # here we created class that help us to save the events for each team in the video
            # and at the end it's will help us to analyze and conclude for future purposes 
    def __init__(self):
        self.first_half_goals = 0
        self.second_half_goals = 0
        self.total_goals = 0
        
        self.total_dangerous_attacks = 0
        self.first_half_dangerous_attacks = 0
        self.second_half_dangerous_attacks = 0
        
        self.total_corners = 0
        
        self.right_corners = 0
        self.left_corners = 0
        
        self.first_half_corners=0
        self.second_half_corners=0

    def get_first_half_goals(self):
        return self.first_half_goals

    def get_second_half_goals(self):
        return self.second_half_goals

    def get_total_goals(self):
        return self.total_goals

    def get_first_half_dangerous_attacks(self):
        return self.first_half_dangerous_attacks

    def get_second_half_dangerous_attacks(self):
        return self.second_half_dangerous_attacks


    def get_dangerous_attacks(self):
        return self.total_dangerous_attacks

    def get_total_corners(self):
        return self.total_corners

    def get_right_corners(self):
        return self.right_corners

    def get_left_corners(self):
        return self.left_corners
    
    def get_first_half_corners(self):
        return self.first_half_corners

    def get_second_half_corners(self):
        return self.second_half_corners

    def update_first_half_goals(self):
        self.update_first_half_dangerous_attacks()
        self.first_half_goals = self.first_half_goals + 1
        self.total_goals = self.total_goals + 1


    def update_second_half_goals(self):
        self.update_second_half_dangerous_attacks()
        self.second_half_goals = self.second_half_goals + 1
        self.total_goals = self.total_goals + 1

    def update_left_corners(self):
        self.left_corners = self.left_corners + 1
        self.total_corners=self.total_corners+1

    def update_right_corners(self):
        self.right_corners = self.right_corners + 1
        self.total_corners=self.total_corners+1
        
    def update_first_half_corners(self):
        self.first_half_corners = self.first_half_corners + 1
  
        
    def update_second_half_corners(self):
        self.second_half_corners = self.second_half_corners + 1
        

    def update_first_half_dangerous_attacks(self):
        self.first_half_dangerous_attacks = self.first_half_dangerous_attacks + 1
        self.total_dangerous_attacks = self.total_dangerous_attacks + 1

    def update_second_half_dangerous_attacks(self):
        self.second_half_dangerous_attacks = self.second_half_dangerous_attacks + 1
        self.total_dangerous_attacks = self.total_dangerous_attacks + 1
    
    def printTeamInfo(self,teamName):
        print("Team",teamName," : ")
        print("Total goals - ",self.get_total_goals())
        print("1st half goals - ",self.get_first_half_goals())
        print("2nd half goals - ",self.get_second_half_goals())

        print("Total dangerous attacks - ",self.get_dangerous_attacks())
        print("1st half dangerous attacks - ",self.get_first_half_dangerous_attacks())
        print("2st half dangerous attacks - ",self.get_second_half_dangerous_attacks())

        print("Total corners - ",self.get_total_corners())
        print("Right corners - ",self.get_right_corners())
        print("Left corners - ",self.get_left_corners())
        
        print("1st half corners - ",self.get_first_half_corners())
        print("2nd half corners - ",self.get_second_half_corners())
        print("=============================")

## Detect frame as corner by image processing

In [ ]:
# all the corners function - based on bgr format
# return true if a right top corner detected in current frame
def rightTopCornerFrameDetect(current_frame, x ,y):
    blue,green,red = current_frame[x][y]
    flag = False
    if blue < 80 and green > 180 and red > 200:  # bgr
        flag = True
    elif blue < 40 and green > 130 and red > 160:
        flag = True
    return flag


# return true if a left top corner detected in current frame
def leftTopCornerFrameDetect(current_frame, x ,y):
    blue,green,red = current_frame[x][y]
    flag = False
    if blue < 80 and green > 180 and red > 200:  # bgr
        flag = True
    elif blue < 40 and green > 130 and red > 160:
        flag = True
    elif blue in range(63, 68) and green > 160 and red > 230:
        flag = True
    return flag


# return true if a left bottom corner detected in current frame
def leftBottomCornerFrameDetect(current_frame, x ,y):
    blue,green,red = current_frame[x][y]
    flag = False
    if blue in (range(38, 42)) and green in (range(111, 114)) and red in (range(104, 107)):  
        flag = True
    elif blue in (range(64, 66)) and green in (range(120, 123)) and red in (range(92, 95)): 
        flag = True
    return flag


# return true if a right bottom corner detected in current frame
def rightBottomCornerFrameDetect(current_frame, x ,y):
    blue,green,red = current_frame[x][y]
    flag = False
    if blue in (range(58,73)) and green in (range(112,127)) and red in(range(104,117)): 
        flag = True
    elif blue == 91 and green in (range(125,127)) and red in (range(102,106)):  
        flag = True
    elif blue in (range(48, 53)) and green in (range(61, 64)) and red in (range(55, 61)):
        flag = True
    return flag


#  global function for updating frame counter and the corner counters
def updateCornerAndFramesStatus(corner_id,flag_value,frames_threshold ,num_frames_list,teamA,teamB,frame,frameID):
    if flag_value is True:
        if num_frames_list[corner_id] < frames_threshold:
            num_frames_list[corner_id] = num_frames_list[corner_id] + 1
    else:
        if num_frames_list[corner_id] == frames_threshold: 
            #here we know that the current frame is a corner event
            # and here we classify the corner side , and the corner's team
           if (halfTimeClassification(frame,frameID) == 1):
                    if(corner_id % 2 == 0):
                        teamA.update_first_half_corners()
                        
                        if(corner_id == 2):
                            teamA.update_right_corners()
                        else: teamA.update_left_corners()
                            
                    else:
                        teamB.update_first_half_corners()
                        if corner_id == 1:
                            teamB.update_right_corners()
                        else: teamB.update_left_corners()
                        
           else: 
                   if(corner_id % 2 == 0):
                        teamB.update_second_half_corners()
                        
                        if(corner_id == 2):
                            teamB.update_right_corners()
                        else: teamB.update_left_corners()
                            
                   else:
                        teamA.update_second_half_corners()
                        if corner_id == 1:
                            teamA.update_right_corners()
                        else: teamA.update_left_corners()
         
        num_frames_list[corner_id] = 0

## Detect frame as suspicions  goal

In [ ]:
def detectGoal(current_frame, x,frameID):
    #Check for current frame if the ball is on the goal line according to bgr formats
    counter = 0
    for i in range(624,639):
        blue, green, red = current_frame[i][x]
        if blue in range(50,100) and green > 159 and red > 180:
            counter = counter+1
# When the current frame has at least one goal line coordinate detected ,
#we checked if the game map has changed due a potential goal or dangerous shot
        lengthLineCounter = 0
        if counter != 0:
            for i in range(550,731):
                blue, green, red = current_frame[576][i]
                if blue > 100 and green > 150 and red > 120 :
                    lengthLineCounter = lengthLineCounter+1
# Sampling the map horizontal line for the frame , 
# and counting its as positive if the colour is green under bgr definition
        if lengthLineCounter >= 100:
            pass
        else:
            counter = 0
# return the counter result according the map status  ,
# for prevent optional mistakes and increasing the event accuracy level
    if counter >= 6:
        counter = -1

    return counter



# Check the map status for the goal detection decision
def funcCheckCounterMap(current_frame,frameID):
      counter=180
      for i in range(550,731):
                blue, green, red = current_frame[576][i]
                if blue > 100 and green > 150 and red > 120 :
                    counter = counter-1
# Sampling the map horizontal line for the frame , and counting how many points on this line are green.

# In order to future define of the event and seperate between a goal to dangerous attack,
# by the period time the map changed.	
      if counter > 150:
          return 1
      elif counter < 10:
          return -13
      else:
          return 0


## Init variables 

In [ ]:
# count for each corner the detected frames
framesDetectedList = [0,0,0,0]
# loading the game video
cap = cv2.VideoCapture("/home/linuxu/Desktop/folder/videoExample.mp4")
# requested starting frame number
startFrame = 3000
# counting the game frames
i = 0

# Index for the goal lists
frameGoal = 0

# the  goal lists for 3 consecutive frames
rightGoalList = [0,0,0]
leftGoalList = [0,0,0]

# Delay parameter,of 150 frames between each suspicious goal event
counterFramesCheckLeft = 0
counterFramesCheckRight = 0

# Counting the map status for every frame  for suspicious goal event
counterCheckMapRight = 0
counterCheckMapLeft = 0

# True if a suspicious goal detected else false
flagGoalRightFrame = False
flagGoalLeftFrame = False

# Responsible for pausing the time between detection the goal event until starting the map check action time
delayCounterRight = 0
delayCounterLeft = 0


cardsCounterDetect=0
cardsCount=0

teamA = Team()
teamB = Team()

## Video capture session

In [ ]:
# running on the video frames until end of the match
while cap.isOpened():
    ret,frame = cap.read()
    if i >= startFrame:
       # operate the card model
       if cardsCounterDetect == 250:
          if detectCard(frame):
                cardsCount=cardsCount+1
                print("Card detected!")
          cardsCounterDetect=0
       cardsCounterDetect=cardsCounterDetect+1

        # 4 functions for the corners detection
        # 0 - detect right top , detect 1 = left top ,  2 detect = right bottom  ,3 detect = left bottom 
       updateCornerAndFramesStatus(0, rightTopCornerFrameDetect(frame, 579, 729), 100, framesDetectedList,teamA,teamB,frame,i)
       updateCornerAndFramesStatus(1, leftTopCornerFrameDetect(frame,578,551),80,framesDetectedList,teamA,teamB,frame,i)
       updateCornerAndFramesStatus(2, rightBottomCornerFrameDetect(frame,681,729),18,framesDetectedList,teamA,teamB,frame,i) 
       updateCornerAndFramesStatus(3,leftBottomCornerFrameDetect(frame,681,551),80,framesDetectedList,teamA,teamB,frame,i)
        # ################# Right goal detection event part
       
       # Assign the frame goal line count to the list 
       rightGoalList[frameGoal] = detectGoal(frame,729,i)
       frameGoal = frameGoal+1
       if frameGoal == 3:
            frameGoal = 0

       if rightGoalList[frameGoal-1] == -1:
            rightGoalList = [0,0,0]

#here we check if the ball is on the goal line in 3 consecutive frames and may be a goal after a few more tests
       if sum(rightGoalList) >= 2:

        
# Check if the the algorithm is not already in a potential goal detection scenario , for creating a new one        
            if flagGoalRightFrame is False:
                counterFramesCheckRight = i + 150
                flagGoalRightFrame = True
                
                
# In case we statrting a new event , the algorithm will get a 150 frames delay
# to make sure the next frames will not start the same process again and lead to a future mistake in detetcion.
            rightGoalList = [0,0,0]

# In case the goal event detection is running,we give a slightly delay of 30 frames for the algorithm
# in order to anylize what is the game map status at the time of the optional event,
# Because it take some time until the field image is ready for the sampling.


       if flagGoalRightFrame is True:
            delayCounterRight = delayCounterRight+1


       if counterFramesCheckRight > i and flagGoalRightFrame is True:
            if delayCounterRight >= 30:
              counterCheckMapRight = funcCheckCounterMap(frame,i) + counterCheckMapRight
            
            #Assign the map status value to a counter in each frame
       else:
        
        # If the map status is more than 20 , the event define as goal 
        
            if counterCheckMapRight >= 20: 
                 print("Right goal detected!, frame = ",i)
                    
#Determine to which team the goal event belongs and on which half time,
#with the following logic(full functions description is in the upper cells) 

                 if halfTimeClassification(frame,i) == 1 :
                    teamA.update_first_half_goals()
                 else: teamB.update_second_half_goals()
                 delayCounterRight = 0
                    
#The map status  condition for dangeroys attack classification part,for relevant tean and half time.

            if counterCheckMapRight < 0:
                print("dangerous attack attempt - RIGHT SIDE")
                if halfTimeClassification(frame,i) == 1 :
                    teamA.update_first_half_dangerous_attacks()
                else: teamB.update_second_half_dangerous_attacks()
                    
#At the end of detetction , all the relevant variables are reinitialized

            flagGoalRightFrame = False
            counterFramesCheckRight = 0
            counterCheckMapRight = 0


                # ############################### Left goal detection event part
 # Assign the frame goal line count to the list
       leftGoalList[frameGoal] = detectGoal(frame,550,i)
       frameGoal = frameGoal+1
       if frameGoal == 3:
            frameGoal = 0

       if leftGoalList[frameGoal-1] == -1:
            leftGoalList = [0,0,0]
#here we check if the ball is on the goal line in 3 consecutive frames and may be a goal after a few more tests

       if sum(leftGoalList) >= 2:
# Check if the the algorithm is not already in a potential goal detection scenario , for creating a new one
        
            if flagGoalLeftFrame is False:
                counterFramesCheckLeft = i+150
                flagGoalLeftFrame = True
                
# In case we statrting a new event , the algorithm will get a 150 frames delay
# to make sure the next frames will not start the same process again and lead to a future mistake in detetcion.
                
            leftGoalList = [0,0,0]

#In case the goal event detection is running,we give a slightly delay of 30 frames for the algorithm
# in order to anylize what is the game map status at the time of the optional event.
#Because it take some time until the field image is ready for the sampling.


       if flagGoalLeftFrame is True:
            delayCounterLeft = delayCounterLeft+1


       if counterFramesCheckLeft > i and flagGoalLeftFrame is True:
            if delayCounterLeft >= 30:

              counterCheckMapLeft = funcCheckCounterMap(frame,i) + counterCheckMapLeft
            
            #Assign the map status value to a counter in each frame             
       else:
        # If the map status is more than 20 , the event define as goal 
        
            if counterCheckMapLeft >= 20:
                 print("Left goal detected! frame = ",i)
                    
#Determine to which team the goal event belongs and on which half time,
#with the following logic(full functions description is in the upper cells)   

                 if halfTimeClassification(frame,i) == 1 :
                     teamB.update_first_half_goals()
                 else: teamA.update_second_half_goals()
                 delayCounterLeft=0
                    
#The map status  condition for dangeroys attack classification part,for relevant tean and half time.     

            if counterCheckMapLeft<0:
                print("dangerous attack attempt - LEFT SIDE")
                if halfTimeClassification(frame,i) == 1 :
                     teamB.update_first_half_dangerous_attacks()
                else: teamA.update_second_half_dangerous_attacks()
                    
#At the end of detetction , all the relevant variables are reinitialized   

            flagGoalLeftFrame = False
            counterFramesCheckLeft = 0
            counterCheckMapLeft = 0


        # representing the picture of the video
       resizeFrame = cv2.resize(frame,(1000,500))
       cv2.imshow('Soccer', resizeFrame)
# #### in case you want to close the video press q
    if cv2.waitKey(10) & 0xFF == ord('q'):
     break
    i = i + 1



# #### closing the video and destroy all the relevant windows
cap.release()
cv2.destroyAllWindows()

## Numerical statistics

In [ ]:
teamA.printTeamInfo('A')
teamB.printTeamInfo('B')
print("You have in the game :",cardsCount,"cards")

## Team statistics display by graph - comparison between 1st to 2nd half events

In [ ]:
def gameEventsPlot(team,teamName):
    partition = ['Total','1st half','2nd half','Total','1st half','2nd half','Total','1st half','2nd half']
    count = [team.get_total_goals(),team.get_first_half_goals(),team.get_second_half_goals(),team.get_dangerous_attacks(),team.get_first_half_dangerous_attacks(),team.get_second_half_dangerous_attacks(),team.get_total_corners(),team.get_first_half_corners(),team.get_second_half_corners()]
    events = ['Goals','Goals','Goals','Dangerous attacks','Dangerous attacks','Dangerous attacks','Corners','Corners','Corners']
    df = pd.DataFrame({'partition': partition,'Quantity': count," ": events})

    sns.set(font_scale=1.5,rc={'figure.figsize':(11.7,8.27)})
    sns.barplot(x=" ", y="Quantity", hue="partition", data=df).set(title= teamName+" statistics")

In [ ]:
gameEventsPlot(teamA,'teamA')

In [ ]:
gameEventsPlot(teamB,'teamB')

## Team corners display by graph - comparison between right to left corner events - it will help to point on which side the team attacks better

In [ ]:
def cornersPlot(team,teamName):
    count = [team.get_left_corners(),team.get_right_corners()]
    events = ['Left corner','Right corner']
    df = pd.DataFrame({'Quantity': count," ": events})

    sns.set(font_scale=1.5,rc={'figure.figsize':(11.7,8.27)})
    sns.barplot(x=" ", y="Quantity", data=df).set(title=teamName+" - corners analysis")

In [ ]:
cornersPlot(teamA,'teamA')

In [ ]:
cornersPlot(teamB,'teamB')

# Game data processing for summary report & conclusions

In [ ]:
#here we init relevant variables for analysis and calcuate game events , and conclude 
# about the game for the report. saving the final report to a file for each team with relevant notes. 

def gameReportAnalysis(team,teamName,opponent):
    if team.get_second_half_dangerous_attacks() == 0:
        goalRateteamSecondHalf=0

    if team.get_first_half_dangerous_attacks() == 0:
        goalRateteamFirstHalf=0

    if team.get_dangerous_attacks() == 0:
        goalRateteamFirstHalf=0


    goalRateteamSecondHalf = team.get_second_half_goals() / (team.get_second_half_dangerous_attacks())
    goalRateteamFirstHalf = team.get_first_half_goals() / (team.get_first_half_dangerous_attacks())
    goalRateTeamTotal = team.get_total_goals() / team.get_dangerous_attacks()

    scoreFirstHalfEvents = team.get_first_half_goals()*2 + team.get_first_half_dangerous_attacks()*1.5 + team.get_first_half_corners()*0.5
    scoreSecondHalfEvents = team.get_second_half_goals()*2 + team.get_second_half_dangerous_attacks()*1.5 + team.get_second_half_corners()*0.5
    attacksRank = (scoreFirstHalfEvents+scoreSecondHalfEvents)
    totalScoreEvents = scoreFirstHalfEvents-scoreSecondHalfEvents
    
    defRank = opponent.get_total_goals()*2 +opponent.get_dangerous_attacks()*1.5+opponent.get_total_corners()
    

    if max(team.get_first_half_goals(),team.get_first_half_dangerous_attacks(),team.get_first_half_corners()) == team.get_first_half_goals():
        elementNameFirstHalf = "goals"

    elif max(team.get_first_half_goals(),team.get_first_half_dangerous_attacks(),team.get_first_half_corners()) == team.get_first_half_dangerous_attacks():
         elementNameFirstHalf = "dangerous attacks"

    else:  elementNameFirstHalf = "corners"

    if max(team.get_second_half_goals(),team.get_second_half_dangerous_attacks(),team.get_second_half_corners()) == team.get_second_half_goals():
        elementNameSecondHalf = "goals"

    elif max(team.get_second_half_goals(),team.get_second_half_dangerous_attacks(),team.get_second_half_corners()) == team.get_second_half_dangerous_attacks():
         elementNameSecondHalf = "dangerous attacks"

    else:  elementNameSecondHalf = "corners"  


    if min(team.get_first_half_goals(),team.get_first_half_dangerous_attacks(),team.get_first_half_corners()) == team.get_first_half_goals():
        minElementNameFirstHalf = "goals"

    elif min(team.get_first_half_goals(),team.get_first_half_dangerous_attacks(),team.get_first_half_corners()) == team.get_first_half_dangerous_attacks():
         minElementNameFirstHalf = "dangerous attacks"

    else:  minElementNameFirstHalf = "corners"

    if min(team.get_second_half_goals(),team.get_second_half_dangerous_attacks(),team.get_second_half_corners()) == team.get_second_half_goals():
        minElementNameSecondHalf = "goals"

    elif min(team.get_second_half_goals(),team.get_second_half_dangerous_attacks(),team.get_second_half_corners()) == team.get_second_half_dangerous_attacks():
         minElementNameSecondHalf = "dangerous attacks"

    else:  minElementNameSecondHalf = "corners"      

    maxElementSecondHalf = max(team.get_second_half_goals(),team.get_second_half_dangerous_attacks(),team.get_second_half_corners())    
    maxElementFirstHalf = max(team.get_first_half_goals(),team.get_first_half_dangerous_attacks(),team.get_first_half_corners())
    maxElementTotal = max(team.get_total_goals(),team.get_dangerous_attacks(),team.get_total_corners())

    minElementSecondHalf = min(team.get_second_half_goals(),team.get_second_half_dangerous_attacks(),team.get_second_half_corners()) 
    minElementFirstHalf =  min(team.get_first_half_goals(),team.get_first_half_dangerous_attacks(),team.get_first_half_corners())
    minElementTotal = min(team.get_total_goals(),team.get_dangerous_attacks(),team.get_total_corners())

    if max(team.get_total_goals(),team.get_dangerous_attacks(),team.get_total_corners()) == team.get_total_goals():
        maxElementNameTotal = "goals"

    elif max(team.get_total_goals(),team.get_dangerous_attacks(),team.get_total_corners()) == team.get_dangerous_attacks():
         maxElementNameTotal = "dangerous attacks"

    else:  maxElementNameTotal = "corners"


    if min(team.get_total_goals(),team.get_dangerous_attacks(),team.get_total_corners()) == team.get_total_goals():
        minElementNameTotal = "goals"

    elif min(team.get_total_goals(),team.get_dangerous_attacks(),team.get_total_corners()) == team.get_dangerous_attacks():
         minElementNameTotal = "dangerous attacks"

    else:  minElementNameTotal = "corners"    
        
    filename = teamName+'_'+str(date.today().strftime("%d_%m_%Y"))
    with open(filename,'w') as f:    
        sys.stdout = f
        if team.get_total_goals() > opponent.get_total_goals():
            print("Your team won in the game!")
        elif opponent.get_total_goals() > team.get_total_goals():
            print("Your team lost in the game!")
        else:
            print("The game ended with a draw!")

        print("General analysis : ")    

        print("Your best parameter was :",maxElementNameTotal, "with amount : ",maxElementTotal)
        print("Your worst parameter was :",minElementNameTotal, "with amount : ",minElementTotal)


        if team.get_right_corners() > team.get_left_corners()+2:
            print("Your team's attacks from the right side had higher odds to lead to a good chance to score.")

        elif team.get_right_corners()+2 < team.get_left_corners(): 
            print("Your team's attacks from the left side had higher odds to lead to a good chance to score.")

        else:  print("Your team's attacks from both sides were balanced , there is an equal odds to score from each of them.")

        if opponent.get_total_goals() > 2:
            print("Your defence game was bad today , you would spend more time on a defense part in the trainings")

        if opponent.get_right_corners() > opponent.get_left_corners():
            print("Your opponent's attacks from the right sides were difficult to stop for your defense players")
        elif opponent.get_right_corners() < opponent.get_left_corners():
            print("Your opponent's attacks from the left sides were difficult to stop for your defense players")


        print("Your team's attack rank was : ",attacksRank,"(above 14 is good , below 8 is not good enough)")
        print("Your team's defense rank was : ",defRank,"(below 10 is good , above 16 is not good enough)")
        print("\n\n")
        print("Analysis games events of between half times : ")
        print("")

        if totalScoreEvents>2:
            print("Your team was better in the first half")
            print("First half data analysis - ")
            print("")
            print("The difference between the half times may happen when the players stamina is low,")
            print("or in case the second half substitutions didn't affect on the game.")
            print()
            print("The most promiment parameter of your team in 1st half was",elementNameFirstHalf , ", the amount : ",maxElementFirstHalf)
            if goalRateteamFirstHalf > 0.40:
                print("Your team's goal rate was good , and the accuracy was : ",goalRateteamFirstHalf*100,"%")
            else:
                print("Your team's goal rate was not good enough , and the accuracy was : ",goalRateteamFirstHalf*100,"%")

        elif totalScoreEvents<-2:
                print("Your team was better in the second half")  
                print("Second half data analysis - ")
                print("")
                print("The difference between the half times may happen when the players stamina is high,")
                print("or in case the second half substitutions brought a positive impact to the game")
                print("The most promiment parameter of your team in 2nd half was ",elementNameSecondHalf , ", the amount : ",maxElementSecondHalf)
                if goalRateteamSecondHalf > 0.40:
                    print("Your team's goal rate was good , and the accuracy was : ",goalRateteamSecondHalf*100,"%")
                else:
                    print("Your team's goal rate was not good enough , and the accuracy was : ",goalRateteamSecondHalf*100,"%")

        else:
              print("Your both half times were balanced")
              print("The small difference between the half times may happen when the players stamina is high,")
              print("or in case the second half substitutions didn't change the game status.")
              print("The most promiment parameter of your team in 1st half was",elementNameFirstHalf , ", the amount : ",maxElementFirstHalf)
              print("The most promiment parameter of your team in 2nd half was ",elementNameSecondHalf , ", the amount : ",maxElementSecondHalf)
              if goalRateTeamTotal > 0.40:
                    print("Your team's goal rate was good , and the accuracy was : ",goalRateTeamTotal*100,"%")
              else:
                    print("Your team's goal rate was not good enough , and the accuracy was : ",goalRateTeamTotal*100,"%")
    f.close()        

In [ ]:
gameReportAnalysis(teamA,'teamA',teamB)

In [ ]:
gameReportAnalysis(teamB,'teamB',teamA)